[Guillaume Genthial blog: Seq2Seq with Attention and Beam Search](https://guillaumegenthial.github.io/sequence-to-sequence.html)

翻译并复现这篇 blog 的目的在于自己一开始手码 attention 就是 transformer 或 Dynamic memory networks 这样的，最基本的 seq2seq attention 机制虽然了解，但没有实现过，对于 attention 的可视化也一直想自己动手去实现。对于 Beam search 怎么实现的也不知道，这也是在实现 transformer 有些疑惑的地方，所以看到这篇 blog 觉得很有必要学一下。

作者在上一篇 blog 介绍了 named entity recognition, 并解释了如何去预测一个词的标签（LSTM+CRF）。这是一个相对简单的任务，但对于像翻译这样的任务会需要更加复杂的系统。你一定听说了，Google 最近在神经机器翻译又有了新的突破，几乎可以达到人类水平，他们发表在 paper: [Google's Multilingual Neural Machine Translation System: Enabling Zero-Shot Translation](https://arxiv.org/abs/1611.04558). 这些新的架构都依赖于一个常用的图模型，叫做编码器-解码器（encoder-decoder or sequence to sequence），其目的是生成一个完整的序列。

### Sequence to Sequence basics
举一个简单的例子， 将 ```how are ypu``` 翻译成法语 ```comment vas tu```.


#### Vanilla Seq2Seq
seq2seq 框架依赖 encoder-decoder 图。其中 encoder 用来编码输入序列，decoder 用来生成目标序列。

#### Encoder
我们的输入是 ```how are you```. 然后将输入序列每个词通过词向量关联成向量 $w\in R^d$(via a lookup table, ```tf.nn.embedding_lookup```). 在我们的case中，有3个词，因此我们的输入可以转换成 $[w_0,w_1,w_2]\in R^{d\times 3}$. 然后我们通过简单的使用一个 LSTM 编码这些向量，并储存隐藏曾最后一层的输出，作为 encoder 的表示 $e$. 每个时间步的隐藏曾输出是 $[e_0,e_1,e_2]$,因此 $e=e_2$

![](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_vanilla_encoder.svg)

### Decoder
现在我们有了一个捕获了输入序列信息的向量 $e$,我们将用它来 word by word 生成目标序列. 将它输入进另一个 LSTM 的cell中: $e$ 作为隐藏层状态，并使用一个特殊的向量 $w_{sos}$ 作为序列的初始输入。

![](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_vanilla_decoder.svg)

LSTM 用来计算下一个隐藏层状态 $h_0\in R^h$, 然后使用一些函数(一般是全连接层) $g:R^h\leftarrow R^V$ 得到 $s_0:=g(h_0)\in R^V$，其中向量 $s_0$ 的维度 size $V$ 是词表的大小。

$$h_0=LSTM(e, w_{sos})$$
$$s_0=g(h_0)$$
$$p_0=softmax(s_0)$$
$$i_0=argmax(p_0)$$

然后使用 softmax 将 $s_0$ 归一化得到一个概率向量 $p_0\in R^V$. 现在 $p_0$ 中的每一个值表示词表中每一个词的概率。我们知道法语单词 "comment" 是 $p_0$ 中概率最大的，上式 $i_0=argmax(p_0)$ $i_0$ 就是 "comment" 在词典中的 index. 然后我们从词向量中得到向量 $w_{i0}=w_{comment}$,然后重复上面的步骤： LSTM 将 $h_0$ 和 $w_{comment}$ 作为输入，并输出概率向量 $p_1$ 用来预测下一个词。

$$h_1=LSTM(h_0, w_{i0})$$
$$s_1=g(h_1)$$
$$p_1=softmax(s_1)$$
$$i_1=argmax(p_1)$$

上述 decoder 的方法就是序列的初始值作为条件，然后对下一个词的概率分布进行建模。原本我们是已整个句子作为条件：
$$P[y_{t+1}|y_1,...,y_t,x_0,x_1,...,x_n]$$
现在转变成类似于 隐马尔可夫模型的形式：
$$P[y_{t+1}|y_t,h_t,e]$$

### Seq2Seq with Attention

上述模型在过去一些年得到改进，尤其是从 attention 中获益很大。 Attention  机制是强制模型在 decoder 中学习注意(focus on) 输入序列中的某一特殊的部分，而不是仅仅依赖于 decoder 的 LSTM 的隐藏状态。应用 attention 机制的一种方式是 paper [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) 中表现的那样。所以我们需要对 decoder 中递归的公式进行一些修改，增加一个新的向量 $c_t$ 来作为 LSTM 的输入。

$$h_t=LSTM(h_{t-1},[w_{i_{t-1}},c_t])$$
$$s_t=g(h_t)$$
$$p_t=softmax(s_t)$$
$$i_t=argmax(p_t)$$

其中 $c_t$ 表示注意力向量(attention or context vector).在 decoder 中的每一步我们需要计算一个 context vector.  
- 第一步：通过一个function  $f(h_{t-1},e_{t'}\rightarrow a_{t'})\in R$ 计算 encoder 中每一个隐藏状态的得分
- 然后对 $a_{t'}$ 归一化得到概率 $\overline a=sofmax(a)$
- 加权求和计算当前时间步的 attention vector $c_t=\sum_{t'=0}^n\overline a_{t'}e_{t'}$

$$a_{t'}=f(h_{t-1},e_{t'})\in R \text{  for all t'}$$
$$\overline a=softmax(a)$$
$$c_t=\sum_{t'=0}^n\overline a_{t'}e_{t'}$$

![](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_attention_mechanism_new.svg)

其中怎么计算当前隐藏状态 $h_{t-1}$ 和 输入序列的隐藏状态序列 $e_1,e_2,..,e_n$ 的atention 权重呢？ 也就是第一步使用什么样的 function f:
$$
f(h_{t-1}, e_{t'}) =
\begin{cases}
h_{t-1}^T e_{t'} & \text{dot}\\
h_{t-1}^T W e_{t'} & \text{general}\\
v^T \tanh \left(W [h_{t-1}, e_{t'}]\right) & \text{concat}\\
\end{cases}$$

对一归一化后的 attention weights $\overline a$ 很好解释。当 decoder 要生成法语单词 ```vas``` 我们希望 $\overline a_{are}$ 接近于1, 而 $\overline a_{how}, \overline a_{you}$ 的权重接近于0. 直觉上告诉我们，在生成法语词 ```vas``` 时，当前 attention vector 大致上与 encoder 中的 ```are``` 的隐藏状态相等。

关于如何将 attention 权重转换成矩阵(rows=input sequence, columns=output sequence).我们需要将英文单词和法语单词进行对齐。除此之外，关于 seq2seq 模型还有很多可说的，具体可参考 paper [NEURAL MACHINE TRANSLATION BY JOINTLY LEARNING TO ALIGN AND TRANSLATE](https://arxiv.org/pdf/1409.0473.pdf),比如对 encoder 使用反向 LSTM 效果会更好。

### Training

> 如果我们在 decoder 的第一步生成词的时候不确定是选择 ```comment``` 或是 ```vas```（这在训练的开始阶段是比较容易发生的）。那样会造成生成的整个序列很乱，模型也很难学到什么东西....

如果我们使用预测的词作为下一个时间步的输入时，错误会积累，并且模型会很少得到正确的分布，导致训练变得很慢或者很难训练。为了加速训练，我们可以采用一个trick，将真实的目标序列 ```(<sos> comment vas tu)``` 作为 decoder LSTM 的输入，然后在此基础上预测下一个词 ```(comment vas tu <eos>)```.

![](https://guillaumegenthial.github.io/assets/img2latex/img2latex_training.svg)

decoder 在每个时间步会得到一个概率的向量 $p_i\in R^V$. 因此对于一个给定的序列 $y_1,...,y_n$ 我们可以将每一步预测得到这个词的概率相乘得到预测整个序列的概率：
$$P(y_1,...,y_m)=\prod_{i=1}^mp_i[y_i]$$

其中 5$p_i[y_i]$ 表示我们在第 i 个时间步得到的概率分布 $p_i$ 中第 $y_i$ 个词的值。当我们的系统非常完美的时候，得到这个序列的概率应该是 1,因此我们可以将这个概率的负对数作为目标函数，最小化这个值来训练我们的神经网络。
$$\begin{align}
-logP(y_1,...,y_m)&=-log\prod_{i=1}^mp_i[y_i]\\
&=-\sum_{i=1}^nlogp_i[y_i]
\end{align}$$

在我们的例子中，它等价于：
$$-logp_1[comment]-logp_2[vas]-logp_3[tu]-logp_4[<eos>]$$
你会意识到这就是个标准的交叉熵：我们实际上就是在最小化目标分布（one-hot向量）和模型得到的预测分布（vector $p_i$）之间的交叉熵。

### Decoding
上面讨论的是，在同一个模型中，我们可以定义不同的行为。在实际中，我们定义了一个特殊的行为用来加速训练。

> 那么在 inference/test 阶段呢？有另一种方式来 decode 序列吗？

测试阶段的 decoding 有两种主要的方法。第一种是在文章开头提到的：贪心解码(greedy decoding).这也是最自然的方式，当前时间步预测的最有可能的词作为下一个时间步的输入。

![](https://guillaumegenthial.github.io/assets/img2latex/seq2seq_vanilla_decoder.svg)



但是我们讨论过，这样会累计错误？

即使是在模型训练阶段使用过的数据，也可能发生错误，比如在 decoding 的第一步更倾向于 ```vas``` 而不是 ```comment```.这会导致整个 decoding 发生错误。

有一个更好的方式进行 decoding,叫做 ```Beam Search```. 我们跟踪 k 中假设（举例 k=5, k 就是 beam size）。在每一个新的时间步，对于这 5 中假设，我们需要 V 个新的 tokens。那么总共就有 5V 个新的假设。然后选择 5 个概率最大的。。依次循环下去。我们定义在 t 时间步的假设为 $H_t$:
$$H_t:=\{(w_1^1,...,w_t^1),....,(w_1^k,....,w_t^k)\}$$

举个栗子 k=2：
$$H_2:=\{(\text{comment vas}),(\text{comment tu})\}$$

一旦每个假设获得 ```<eos>```,我们就返回 k 个假设中概率最大的一个。

> 如果我们使用 beam search,在第一步可能出现偏差，在下一步也能被修正。因此在 beam 中我们能会的最好的假设。

### 结论
在这篇文章中，我们介绍了 seq2seq 的概念。我们知道了在训练阶段和测试阶段的 decoding 是不一样的。我们提到了两种方法 greedy 和 beam search. 虽然 beam search 在结果上表现更好，但是它也不是完美的，依然会存在 exposure bias.在训练阶段，模型也一直存在 exposure errors.同样，beam search也存在 loss-evalution mismatch 的问题。因为我们的模型优化的是单词级别（token-level）的交叉熵，但是我们感兴趣的是重构整个sentence.

关于 seq2seq 的经典文章：  
- [Sequence to Sequence Learning with Neural Networks](https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf)  
- [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473)  
- [Effective Approaches to Attention-based Neural Machine Translation](https://arxiv.org/abs/1508.04025)  
- [Neural Machine Translation in Linear Time](https://arxiv.org/abs/1610.10099)  
- [Convolutional Sequence to Sequence Learning](https://arxiv.org/abs/1705.03122) 
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)

更多前沿文章来解决一些 limitations:
- [An Actor-Critic Algorithm for sequence prediction](https://arxiv.org/pdf/1607.07086.pdf)  
- [Sequence-to-Sequence Learning as Beam-Search Optimization](https://arxiv.org/pdf/1606.02960.pdf)  
- [Six Challenges for Neural Machine Translation](https://arxiv.org/pdf/1706.03872.pdf)  
- [Professor Forcing: A New Algorithm for Training Recurrent Networks](https://arxiv.org/abs/1610.09038)  
